# Windstorm tracks and footprints derived from reanalysis over Europe between 1940 to present: Windstorm statistics

Production date: 2026-MM-DD

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Produced by: Olivier Burggraaff (National Physical Laboratory).

## 🌍 Use case: Use case listed here in full 

## ❓ Quality assessment question
* **In most cases there should be one question listed here in bold**

**‘Context paragraph’ (no title/heading)** - a very short introduction before the assessment statement describing approach taken to answer the user question. One or two key references could be useful, if the assessment summarises literature.

**Background**

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Finding 1
* Finding 2
* Finding 3
* etc
```

## 📋 Methodology

* Internal consistency: TempestExtremes vs TRACK/Hodges? (read papers)
* 

A ‘free text’ introduction to the data analysis steps or a description of the literature synthesis, with a justification of the approach taken, and limitations mentioned. **Mention which CDS catalogue entry is used, including a link, and also any other entries used for the assessment**.

[Windstorm tracks and footprints derived from reanalysis over Europe between 1940 to present
](https://doi.org/10.24381/bf1f06a9)

Hodges (TRACK) algorithm [[Hoskins+02](https://doi.org/10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2), [Hodges+99](https://doi.org/10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2), [Hodges+95](https://doi.org/10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2)]

TempestExtremes [[Ullrich+21](https://doi.org/10.5194/gmd-14-5023-2021), [Ullrich+17](https://doi.org/10.5194/gmd-10-1069-2017)]

**Note:** This notebook is currently just a brain-dump in anticipation of starting the actual quality assessment at a later stage.

E.g. 'The analysis and results are organised in the following steps, which are detailed in the sections below:' 

**[](section-setup)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-download)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-analysis)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-results)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-setup)=
### 1. Code setup
```{note}
This notebook uses [earthkit](https://github.com/ecmwf/earthkit) for
downloading ([earthkit-data](https://github.com/ecmwf/earthkit-data))
and visualising ([earthkit-plots](https://github.com/ecmwf/earthkit-plots)) data.
Because earthkit is in active development, some functionality may change after this notebook is published.
If any part of the code stops functioning, please raise an issue on our GitHub repository so it can be fixed.
```

#### Imports

In [ ]:
# Input / Output
from pathlib import Path
import earthkit.data as ekd
import warnings

# General data handling
import numpy as np
np.seterr(divide="ignore")  # Ignore divide-by-zero warnings
import pandas as pd
import geopandas as gpd
from functools import partial, reduce
from operator import and_ as bitwise_and
from dask.array.core import PerformanceWarning
warnings.simplefilter(action="ignore", category=PerformanceWarning)

# Visualisation
import earthkit.plots as ekp
from earthkit.plots.styles import Style
import matplotlib.pyplot as plt
plt.rcParams["grid.linestyle"] = "--"
plt.rcParams["figure.constrained_layout.use"] = True
plt.rcParams["figure.labelweight"] = "bold"
plt.rcParams["figure.titleweight"] = "bold"
from matplotlib.colors import LogNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.transforms import Affine2D
import cartopy
import shapely
from shapely.geometry import Polygon, MultiPolygon
import cmcrameri as cmc
from tqdm import tqdm  # Progress bars

# Visualisation in Jupyter book -- automatically ignored otherwise
try:
    from myst_nb import glue
except ImportError:
    glue = None

# Type hints
from typing import Callable, Iterable, Optional
from cartopy.mpl.geoaxes import GeoAxes

#### Helper functions

##### Data (pre-)processing

The following cell contains some pre-defined constants for convenience,
such as a list of variable names in the data:

In [ ]:
# Data
TRACKING_ALGORITHMS = ["hodges", "tempest_extremes"]
NUTS_LEVELS         = [0, 1, 2]

# Columns in data
SELECTION_COLUMNS   = ["algorithm", "threshold", "year",]
VARIABLES           = ["storms_number", "mean_wind_gust", "ssi", "normalised_ssi", "area_ratio", "wind_gust_ratio",]
NUTS_VARIABLES      = ["region", "country_code", "NUTS_level", "NUTS_name",]

# Organisation of data
INDEX_COLUMNS       = ["algorithm", "threshold", "region", "year",]

The following functions select data from a (Geo)DataFrame according to one or multiple conditions,
e.g. a specific year and tracking algorithm:

In [ ]:
def _select_from_column(df: pd.DataFrame, col: str, val) -> pd.Series:
    """
    Create a True/False mask for one column `col` in a (Geo)DataFrame `df`
    where the value is `val`.
    Checks for iterable/non-iterable values and switches between .isin and == accordingly.
    """
    # Check for single / multiple values
    if df[col].dtype == "O":  # String column
        if isinstance(val, str):
            # Value is str -> Single value
            iterable_val = False
        elif isinstance(val, Iterable):
            # Value is Iterable but not str -> assume Iterable[str]
            iterable_val = True
        else:
            raise ValueError(f"Cannot parse value of type `{type(val)}` for column `{col}`.")
    else:  # Non-string column, assume number or similar
        iterable_val = isinstance(val, Iterable)

    # Create True/False mask
    selection = (df[col].isin(val)) if iterable_val else (df[col] == val)
    return selection

def select_data(df: pd.DataFrame, **kwargs):
    """
    Select data from a given (Geo)DataFrame `df` according to any number of conditions,
    matching its column names.
    """
    # Create masks for all col/val pairs in kwargs, then combine with & (bitwise and)
    selection_all = [_select_from_column(df, col, val) for col, val in kwargs.items()]
    selection = reduce(bitwise_and, selection_all)

    # Apply and return
    df_selection = df[selection]
    return df_selection

##### Visualisation

In [ ]:
# Geometry / Projection
EPSG = 3035
CRS = cartopy.crs.epsg(EPSG)
MAP_KWARGS={"projection": CRS,
            "xlim": (0.25e7, 0.74e7), "ylim": (1.3e6, 5.5e6),  # Continental Europe
           }

# Define styles for land, borders, etc.
STYLE_LAND            = {"zorder": 1, "facecolor": "#ccced1", "edgecolor": "none"}
STYLE_COASTLINE       = {"zorder": 3, "edgecolor": "black", "linewidth": 1}
STYLE_NATIONAL_BORDER = {"zorder": 3, "edgecolor": "black", "linewidth": 0.5}
STYLE_CHOROPLETH      = {"zorder": 2, "edgecolor": STYLE_NATIONAL_BORDER["edgecolor"], "linewidth": STYLE_NATIONAL_BORDER["linewidth"]/5}

In [ ]:
_style_footprint = {"cmap": plt.cm.cividis, "vmin": 0, "vmax": 40}

styles = {
    "footprint": Style(**_style_footprint),
}

# Apply general settings
for style in styles.values():
    style.normalize = False

In [ ]:
def _glue_or_show(fig: plt.Figure, glue_label: Optional[str]=None) -> None:
    """
    If `glue` is available, glue the figure using the provided label.
    If not, display the figure in the notebook.
    """
    try:
        glue(glue_label, fig, display=False)
    except TypeError:
        plt.show()
    finally:
        plt.close()

def _add_textbox_to_subplots(text: str, *axs: Iterable[plt.Axes | ekp.Subplot], right=False) -> None:
    """ Add a text box to each of the specified subplots. """
    # Get the plt.Axes for each ekp.Subplot
    axs = [subplot.ax if isinstance(subplot, ekp.Subplot) else subplot for subplot in axs]

    # Set up location
    x = 0.95 if right else 0.05
    horizontalalignment = "right" if right else "left"

    # Add the text
    for ax in axs:
        ax.text(x, 0.95, text, transform=ax.transAxes,
        horizontalalignment=horizontalalignment, verticalalignment="top",
        bbox={"facecolor": "white", "edgecolor": "black", "boxstyle": "round",
              "alpha": 1})

The following cell contains functions for displaying Shapely geometries (e.g. a country shape) next to a plot title:

In [ ]:
def _iter_polygons(geom):
    """Yield polygon parts from Polygon/MultiPolygon; try buffer(0) for non-polygon inputs."""
    if isinstance(geom, Polygon):
        yield geom
    elif isinstance(geom, MultiPolygon):
        for g in geom.geoms:
            yield g
    else:
        g = geom.buffer(0)  # attempt to polygonize (e.g., fix invalid rings)
        if isinstance(g, Polygon):
            yield g
        elif isinstance(g, MultiPolygon):
            for p in g.geoms:
                yield p
        else:
            raise ValueError("Geometry is not polygonal and cannot be polygonized.")

def _polys_to_path(polys, bounds=None):
    """
    Build a single compound Path from a list of polygons.
    Normalize to unit box using `bounds=(minx,miny,maxx,maxy)` if provided.
    """
    if bounds is None:
        u = shapely.unary_union(list(polys))
        minx, miny, maxx, maxy = u.bounds
    else:
        minx, miny, maxx, maxy = bounds

    w = max(maxx - minx, 1e-12)
    h = max(maxy - miny, 1e-12)

    vertices = []
    codes = []
    for poly in polys:
        # exterior
        ex = np.asarray(poly.exterior.coords)
        ex_norm = np.column_stack(((ex[:, 0] - minx) / w, (ex[:, 1] - miny) / h))
        vertices += ex_norm.tolist()
        codes    += [Path.MOVETO] + [Path.LINETO] * (len(ex_norm) - 2) + [Path.CLOSEPOLY]

        # holes
        for ring in poly.interiors:
            ri = np.asarray(ring.coords)
            ri_norm = np.column_stack(((ri[:, 0] - minx) / w, (ri[:, 1] - miny) / h))
            vertices += ri_norm.tolist()
            codes    += [Path.MOVETO] + [Path.LINETO] * (len(ri_norm) - 2) + [Path.CLOSEPOLY]

    return Path(vertices, codes), (minx, miny, maxx, maxy)

def shapely_to_patch(geom, *, bounds=None, facecolor="#d9d9d9", edgecolor="#555", lw=0.4,
                     transform=None, clip_on=False, zorder=10, alpha=1.0):
    """
    Convert any polygonal geometry to a PathPatch normalized to `bounds`.
    If `bounds` is None, bounds of the geometry are used (not recommended when overlaying).
    """
    polys = list(_iter_polygons(geom))
    path, _ = _polys_to_path(polys, bounds=bounds)
    patch = PathPatch(
        path, facecolor=facecolor, edgecolor=edgecolor, linewidth=lw,
        transform=transform, clip_on=clip_on, zorder=zorder, alpha=alpha
    )
    return patch

def add_vector_icon_next_to_title(
    ax,
    geom,
    geom_base=None,
    where="right",
    size=0.1,
    *,
    # styling
    facecolor="#2e6da4", edgecolor="#1f3f67", lw=0.5, alpha=1.0,
    base_facecolor="#cfd8e3", base_edgecolor="#4c5c74", base_lw=0.4, base_alpha=1.0,
    # placement & layering
    y_anchor=1.02, x_anchor_right=0.93, x_anchor_left=0.07,
    zorder_base=10, zorder_geom=11,
    # behavior
    to_figure=False, clip_on=False, pad=0.0,
):
    """
    Place a vector icon near the title using the shapely_to_patch() helper.

    Parameters
    ----------
    ax : matplotlib.axes.Axes
        Target axes (used for placement and for figure access).
    geom : shapely geometry
        The highlighted geometry to draw (e.g., Zeeland).
    geom_base : shapely geometry or None
        Optional base geometry to draw behind `geom` (e.g., Netherlands).
    where : {"right","left"}
        Icon position relative to the title area of this axes.
    size : float
        Icon width/height in axes (or figure) coordinates.
    facecolor, edgecolor, lw, alpha : styling for the highlighted geom.
    base_facecolor, base_edgecolor, base_lw, base_alpha : styling for base geom.
    y_anchor : float
        Vertical anchor above the axes (axes/figure coords depending on `to_figure`).
    x_anchor_right, x_anchor_left : float
        Horizontal anchors for right/left placement (axes/figure coords).
    zorder_base, zorder_geom : int
        Z-orders for base and highlight geometries.
    to_figure : bool
        If True, draw in figure coordinates (never clipped by any axes).
        If False (default), draw in axes coordinates.
    clip_on : bool
        If False (default), allow icon to extend beyond axes.
    pad : float
        Small offset added toward the right or left (same coordinate system as placement).
    """
    # Decide anchor
    x_anchor = (x_anchor_right if where == "right" else x_anchor_left) + (pad if where == "right" else -pad)

    # Bounds for consistent normalization (preserve relative scale/position)
    if geom_base is not None:
        import shapely
        union_bounds = shapely.unary_union([geom_base, geom]).bounds
    else:
        union_bounds = None  # falls back to geom's own bounds

    # Build the placement transform in axes or figure coordinates
    base_transform = ax.figure.transFigure if to_figure else ax.transAxes
    trans = (Affine2D()
             .scale(size, size)
             .translate(x_anchor - size/2, y_anchor - size/2)) + base_transform

    # Construct patches using shared bounds
    patches = []

    if geom_base is not None:
        p_base = shapely_to_patch(
            geom_base,
            bounds=union_bounds,
            facecolor=base_facecolor, edgecolor=base_edgecolor,
            lw=base_lw, alpha=base_alpha,
            transform=trans, clip_on=clip_on, zorder=zorder_base
        )
        patches.append(p_base)

    p_geom = shapely_to_patch(
        geom,
        bounds=union_bounds,
        facecolor=facecolor, edgecolor=edgecolor,
        lw=lw, alpha=alpha,
        transform=trans, clip_on=clip_on, zorder=zorder_geom
    )
    patches.append(p_geom)

    # Attach patches to the appropriate container
    container = ax.figure if to_figure else ax
    for p in patches:
        container.add_artist(p)

    return tuple(patches)

In [ ]:
def decorate_fig(fig: ekp.Figure, *, title: Optional[str]="") -> None:
    """ Decorate an earthkit figure with land, coastlines, etc. """
    # Add progress bar because individual steps can be very slow for large plots
    with tqdm(total=4, desc="Decorating", leave=False) as progressbar:
        fig.land()
        progressbar.update()
        fig.coastlines()
        progressbar.update()
        # fig.borders()
        # progressbar.update()
        fig.gridlines(linestyle=plt.rcParams["grid.linestyle"])
        progressbar.update()
        fig.title(title)
        progressbar.update()

def decorate_fig(axs: Iterable[GeoAxes], *, title: Optional[str]="") -> None:
    """ Decorate a cartopy/matplotlib figure with land, coastlines, etc. """
    # Will be replaced by preceding (earthkit) function when earthkit-plots supports choropleth maps
    try:  # Ravel if numpy array
        axs = axs.ravel()
    except AttributeError:
        pass

    # Apply each decoration in order
    for ax in axs:
        ax.add_feature(cartopy.feature.LAND,      **STYLE_LAND)
        ax.add_feature(cartopy.feature.COASTLINE, **STYLE_COASTLINE)
        ax.add_feature(cartopy.feature.BORDERS,   **STYLE_NATIONAL_BORDER)

In [ ]:
def plot_scatter_by_variable_and_nuts(df: pd.DataFrame, *,
                                      glue_label: Optional[str]=None) -> None:
    """
    Plot a list of variables in one DataFrame, with one-to-one comparisons
    between the two tracking algorithms.
    Hard-coded for VARIABLES, NUTS_LEVELS, TRACKING_ALGORITHMS.
    """
    # Move "year" to data column for this visualisation
    df = df.reset_index(level=-1)

    # Create figure
    nrows, ncols = len(VARIABLES), len(NUTS_LEVELS)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex="row", sharey="row", figsize=(4*ncols, 4*nrows), squeeze=False)

    # Loop over rows: variables
    for ax_row, var in zip(axs, VARIABLES):
        # Loop over columns: NUTS levels
        for ax, nuts in zip(ax_row, NUTS_LEVELS):
            # Select data
            df_here = df[df["NUTS_level"] == nuts]

            # Split by tracking algorithm
            x, y = df_here.loc["hodges"], df_here.loc["tempest_extremes"]

            # Loop over thresholds: shape
            for marker, threshold in zip("ovsX", x.index.levels[0]):
                x_here, y_here = x.loc[threshold, :, :], y.loc[threshold, :, :]

                ax.scatter(x_here[var], y_here[var], marker=marker, c=x_here["year"], cmap=plt.cm.cividis, alpha=0.7)

    # Decorate panels
    for ax in axs.ravel():
        # Highlight diagonal
        ax.axline((0, 0), slope=1, color=plt.rcParams["grid.color"], linewidth=2, linestyle="-")

        # Panel shape
        ax.set_aspect("equal", adjustable="box")
        # limits left free to allow for scatter points around 0

    # Label top row / left column
    for ax, nuts in zip(axs[0], NUTS_LEVELS):
        ax.set_title(f"NUTS level {nuts}")
    for ax, var in zip(axs[:, 0], VARIABLES):
        ax.set_ylabel(var)    

    # Decorate figure
    fig.suptitle("Point-by-point comparison across the dataset")
    fig.supxlabel("Hodges/TRACK")
    fig.supylabel("TempestExtremes")
    fig.align_labels()

    # Display
    plt.show()
    plt.close()

In [ ]:
def plot_choropleth_by_algorithm(df: gpd.GeoDataFrame, col: str, *,
                                 year: int, threshold: float, NUTS_level: int,  # Selection
                                 glue_label: Optional[str]=None) -> None:
    """
    Plot data from column `col` in GeoDataFrame `df` in a choropleth map.
    Compare between both algorithms for one year / threshold / NUTS level.
    """
    # Select data
    df_sel  = df.loc[:, threshold, :, year]
    df_nuts = select_data(df_sel, NUTS_level=NUTS_level)

    # Create figure
    ncols = len(TRACKING_ALGORITHMS) + 1
    fig, axs = plt.subplots(nrows=1, ncols=ncols, figsize=(4*ncols, 4), squeeze=False, subplot_kw=MAP_KWARGS)

    # Plot variable for each algorithm
    for j, alg in enumerate(TRACKING_ALGORITHMS):
        # Select data, panel for this algorithm
        df_alg = df_nuts.loc[alg]
        ax = axs[0, j]

        # Plot
        choro = df_alg.plot.geo(col, ax=ax,
                                vmin=0, vmax=20,
                                legend=True, legend_kwds={"location": "bottom", "label": col},
                                cmap=plt.cm.cividis.resampled(8),
                                **STYLE_CHOROPLETH)
        ax.set_title(alg)

    # Plot difference
    df_diff = df_nuts.loc["difference"]
    ax = axs[0, -1]
    choro = df_diff.plot.geo(col, ax=ax,
                        vmin=-10, vmax=10,
                        legend=True, legend_kwds={"location": "bottom", "label": "Δ "+col},
                        cmap=plt.cm.PuOr_r.resampled(11),
                        **STYLE_CHOROPLETH)
    ax.set_title("Difference")

    # Decorate
    decorate_fig(axs)

    # Show result
    plt.show()
    plt.close()

(section-download)=
### 2. Download data

#### CDS: Windstorm summary indicators
We download the full windstorm summary indicator table from the
[sis-european-wind-storm-reanalysis](https://doi.org/10.24381/bf1f06a9)
CDS entry.
Because this is a simple table,
the data are loaded into a Pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).
Geographical information will be added to this table in the next subsection.

In [ ]:
dataset = "sis-european-wind-storm-reanalysis"

In [ ]:
request = {
    "product": "windstorm_summary_indicator",
    "variable": "all",
    "tracking_algorithm": ["hodges", "tempestextremes"],
    # Note that we do not use the TRACKING_ALGORITHMS constant here because the CDS
    # does not have an _ in `tempestextremes` but the resulting table does.
}

In [ ]:
data = ekd.from_source("cds", dataset, request)
data = data.to_pandas()
data

#### Ancillary: NUTS shapefiles
The summary table is organised by NUTS
(Nomenclature of Territorial Units for Statistics)
level 0, 1, and 2
region.
To visualise these areas,
we need the corresponding shapefiles,
which can be retrieved from [GISCO](https://ec.europa.eu/eurostat/web/gisco/geodata/statistical-units/territorial-units-statistics).
We load these data into a GeoPandas [GeoDataFrame](https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoDataFrame.html):

In [ ]:
# Set up GISCO URL and request
base = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/"
fname = "NUTS_RG_20M_2021_3035_LEVL_{level}.geojson"
# RG: Polygons
# 20M: 1:20 Million scale
# 2021: Year
# 3035: EPSG CRS (projection)
# LEVEL: Iterate over NUTS_LEVELS (0, 1, 2)
urls = [base + "/" + fname.format(level=level) for level in NUTS_LEVELS]

# Download data from GISCO
nuts_gdf = [gpd.read_file(url) for url in urls]

# Merge NUTS levels into one table
nuts_gdf = gpd.GeoDataFrame(pd.concat(nuts_gdf))
nuts_gdf = nuts_gdf.rename(columns={"NUTS_ID": "region"})

The NUTS shapes are merged into the CDS data:

In [ ]:
# Merge NUTS polygons into data
data = nuts_gdf.merge(data, on="region")

# Rename some columns for ease of use
data = data.rename(columns={"LEVL_CODE": "NUTS_level", "CNTR_CODE": "country_code", "NAME_LATN": "NUTS_name"})

# Filter only desired columns, in pre-set order
data = data[[*SELECTION_COLUMNS, *NUTS_VARIABLES, *VARIABLES, "geometry",]]

# Display result
data

#### Re-index
Lastly, we re-index the data by tracking algorithm, threshold, region, and year, for easier subsampling in the analysis later:

In [ ]:
# Re-index data for easier selection
data = data.set_index(INDEX_COLUMNS)
    
# Display result
data

#### Difference between algorithms
Some of the following analysis will focus on the per-point difference between the Hodges and TempestExtremes algorithms.
Here, we calculate this difference and append it to the GeoDataFrame as a third entry along the `algorithm` index:

In [ ]:
# Setup: Copy DataFrame format
difference = data.loc["hodges"].copy()

# Calculate difference
difference[VARIABLES] = data.loc["hodges"][VARIABLES] - data.loc["tempest_extremes"][VARIABLES]

# Append to existing DataFrame, convert back to GeoDataFrame
difference = pd.concat({"difference": difference}, names=["algorithm"])  # Add `algorithm` index level to `difference`
data = pd.concat([data, difference])  # Append to existing data (converts to regular DataFrame)
data = gpd.GeoDataFrame(data)  # Convert to GeoDataFrame again

# Display result
data

(section-analysis)=
### 3. Analysis
Split by method for now because easier to experiment.
Final analysis should probably be by topic rather than by visualisation.
(time series)

#### Tabular comparison

In [ ]:
#

#### Scatter plot comparison

In [ ]:
plot_scatter_by_variable_and_nuts(data)

#### Time series comparison

In [ ]:
from matplotlib.patches import PathPatch

COLOURS_DATA = "#0077bb", "#ee7733"

df = data.copy()
NUTS2_region = "NL34"
threshold = 0.0

"""
Plot a list of variables in one DataFrame, with time series comparisons
between the two tracking algorithms.
Hard-coded for VARIABLES, NUTS_LEVELS, TRACKING_ALGORITHMS.
Columns show one location at incremental NUTS levels (e.g. NL -> NUTS3 -> NUTS34).
For now: one threshold (0.0).
"""
# Generate NUTS subdivisions
assert len(NUTS2_region) == 4, f"Please provide a NUTS2 region with argument `NUTS2_region`. Current value {NUTS2_region} is not understood."
NUTS_region_here = [NUTS2_region[:2], NUTS2_region[:3], NUTS2_region]

# Create figure
nrows, ncols = len(VARIABLES), len(NUTS_LEVELS)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex="row", sharey="row", figsize=(4*ncols, 4*nrows), squeeze=False)

# Loop over rows: variables
for ax_row, var in zip(axs, VARIABLES):
    # Loop over columns: NUTS regions
    for ax, nuts in zip(ax_row, NUTS_region_here):
        # Select data
        df_here = df.loc[:, threshold, nuts]

        # Loop over tracking algorithms
        for alg, c in zip(TRACKING_ALGORITHMS, COLOURS_DATA):
            df_here.loc[alg][var].plot.line(ax=ax, color=c, alpha=0.6, linewidth=2, label=alg)

# Label top row / left column
for ax, nuts in zip(axs[0], NUTS_region_here):
    # Extract NUTS area name from data table
    name = df.loc[:, :, nuts]["NUTS_name"].iloc[0]
    ax.set_title(f"{name} ({nuts})")

    # Add shape
    geometry = df.loc[:, :, nuts]["geometry"].iloc[0]
    geometry_base = df.loc[:, :, NUTS_region_here[0]]["geometry"].iloc[0]
    add_vector_icon_next_to_title(ax, geometry, geometry_base)
    # inset = ax.inset_axes([0, 1, 0.1, 0.1], xticks=[], yticks=[])
    
for ax, var in zip(axs[:, 0], VARIABLES):
    ax.set_ylabel(var)    

# Decorate figure
fig.align_labels()

# Display
plt.show()
plt.close()

#### Geospatial comparison
Note: Choropleth maps are not supported in earthkit-plots yet, but will be soon. See GitHub issue [#160](https://github.com/ecmwf/earthkit-plots/issues/160).

In [ ]:
for NUTS_level in NUTS_LEVELS:
    plot_choropleth_by_algorithm(data, "storms_number", year=2024, threshold=0.0, NUTS_level=NUTS_level)

In [ ]:
# For earthkit-plots issue #160
# data_nuts[selection].to_file("choropleth_example.geojson", driver='GeoJSON')

(section-results)=
### 5. Results

#### Results Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included). 

If this is the **results section**, we expect the final plots to be created here with a description of how to interpret them, and what information can be extracted for the specific use case and user question. The information in the 'quality assessment statement' should be derived here. 

## ℹ️ If you want to know more

### Key resources

Code libraries used:
* [GeoPandas](https://geopandas.org/en/stable/)
* [earthkit](https://github.com/ecmwf/earthkit)
  * [earthkit-data](https://github.com/ecmwf/earthkit-data)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)

### References
[[Hoskins+02](https://doi.org/10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2)] B. J. Hoskins and K. I. Hodges, ‘New Perspectives on the Northern Hemisphere Winter Storm Tracks’, Journal of the Atmospheric Sciences, vol. 59, no. 6, pp. 1041–1061, Mar. 2002, doi: 10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2.

[[Hodges+99](https://doi.org/10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2)] K. I. Hodges, ‘Adaptive Constraints for Feature Tracking’, Monthly Weather Review, vol. 127, no. 6, pp. 1362–1373, June 1999, doi: 10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2.

[[Hodges+95](https://doi.org/10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2)] K. I. Hodges, ‘Feature Tracking on the Unit Sphere’, Monthly Weather Review, vol. 123, no. 12, pp. 3458–3465, Dec. 1995, doi: 10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2.

[[Ullrich+21](https://doi.org/10.5194/gmd-14-5023-2021)] P. A. Ullrich, C. M. Zarzycki, E. E. McClenny, M. C. Pinheiro, A. M. Stansfield, and K. A. Reed, ‘TempestExtremes v2.1: a community framework for feature detection, tracking, and analysis in large datasets’, Geoscientific Model Development, vol. 14, no. 8, pp. 5023–5048, Aug. 2021, doi: 10.5194/gmd-14-5023-2021.

[[Ullrich+17](https://doi.org/10.5194/gmd-10-1069-2017)] P. A. Ullrich and C. M. Zarzycki, ‘TempestExtremes: a framework for scale-insensitive pointwise feature tracking on unstructured grids’, Geoscientific Model Development, vol. 10, no. 3, pp. 1069–1090, Mar. 2017, doi: 10.5194/gmd-10-1069-2017.